**Single table column transformation (col names)**

In [0]:
dbutils.fs.ls('mnt/silver/SalesLT/')

[FileInfo(path='dbfs:/mnt/silver/SalesLT/Address/', name='Address/', size=0, modificationTime=1742289097000),
 FileInfo(path='dbfs:/mnt/silver/SalesLT/Customer/', name='Customer/', size=0, modificationTime=1742289117000),
 FileInfo(path='dbfs:/mnt/silver/SalesLT/CustomerAddress/', name='CustomerAddress/', size=0, modificationTime=1742289129000),
 FileInfo(path='dbfs:/mnt/silver/SalesLT/Product/', name='Product/', size=0, modificationTime=1742289140000),
 FileInfo(path='dbfs:/mnt/silver/SalesLT/ProductCategory/', name='ProductCategory/', size=0, modificationTime=1742289151000),
 FileInfo(path='dbfs:/mnt/silver/SalesLT/ProductDescription/', name='ProductDescription/', size=0, modificationTime=1742289161000),
 FileInfo(path='dbfs:/mnt/silver/SalesLT/ProductModel/', name='ProductModel/', size=0, modificationTime=1742289172000),
 FileInfo(path='dbfs:/mnt/silver/SalesLT/ProductModelProductDescription/', name='ProductModelProductDescription/', size=0, modificationTime=1742289182000),
 FileInf

In [0]:
dbutils.fs.ls('mnt/gold/')

[]

In [0]:
df = spark.read.format('delta').load('/mnt/silver/SalesLT/Address/')
display(df)

In [0]:
from pyspark.sql.functions import col

def rename_columns_to_snake_case(df):
    """
    Convert column names from PascalCase or camelCase to snake_case in a PySpark DataFrame.

    Args:
        df (DataFrame): The input DataFrame with columns to be renamed.

    Returns:
        DataFrame: A new DataFrame with column names converted to snake_case.
    """
    # Get the list of column names
    column_names = df.columns

    # Dictionary to hold old and new column name mappings
    rename_map = {}

    for old_col_name in column_names:
        # Convert column name from PascalCase or camelCase to snake_case
        new_col_name = ''.join([
            ('_' + char.lower()) if (
                char.isupper() and idx > 0 and not old_col_name[idx - 1].isupper()
            ) else char.lower()
            for idx, char in enumerate(old_col_name)
        ]).lstrip('_')

        # Avoid renaming to an existing column name
        if new_col_name in rename_map.values():
            raise ValueError(f"Duplicate column name found after renaming: '{new_col_name}'")

        # Map the old column name to the new column name
        rename_map[old_col_name] = new_col_name

    # Rename columns using the mapping
    for old_col_name, new_col_name in rename_map.items():
        df = df.withColumnRenamed(old_col_name, new_col_name)

    return df

# Get the list of table names from the Silver directory
try:
    table_names = [i.name.split('/')[0] for i in dbutils.fs.ls('mnt/silver/SalesLT/') if i.isDir()]
except Exception as e:
    print(f"Error accessing the Silver directory: {str(e)}")
    table_names = []

# Process each table
for name in table_names:
    try:
        path = f'/mnt/silver/SalesLT/{name}'
        print(f"Processing table: {path}")
        df = spark.read.format('delta').load(path)

        # Rename columns to snake_case
        df = rename_columns_to_snake_case(df)

        # Output path for the Gold layer
        output_path = f'/mnt/gold/SalesLT/{name}/'
        df.write.format('delta').mode('overwrite').save(output_path)
        print(f"Successfully processed and saved: {output_path}")

        display(df)
    except Exception as e:
        print(f"Error processing table '{name}': {str(e)}")


Processing table: /mnt/silver/SalesLT/Address
Successfully processed and saved: /mnt/gold/SalesLT/Address/


address_id address_line1 address_line2 city state_province country_region postal_code date 9 8713 Yosemite Ct. null Bothell Washington United States 98011 2006-07-01 11 1318 Lasalle Street null Bothell Washington United States 98011 2007-04-01 25 9178 Jumping St. null Dallas Texas United States 75201 2006-09-01 28 9228 Via Del Sol null Phoenix Arizona United States 85004 2005-09-01 32 26910 Indela Road null Montreal Quebec Canada H1Y 2H5 2006-08-01 185 2681 Eagle Peak null Bellevue Washington United States 98004 2006-09-01 297 7943 Walnut Ave null Renton Washington United States 98055 2006-08-01 445 6388 Lake City Way null Burnaby British Columbia Canada V5A 3A6 2006-09-01 446 52560 Free Street null Toronto Ontario Canada M4B 1V7 2005-08-01 447 22580 Free Street null Toronto Ontario Canada M4B 1V7 2006-08-01 448 2575 Bloor Street East null Toronto Ontario Canada M4B 1V6 2007-08-01 449 Station E null Chalk Riber Ontario Canada K0J 1J0 2005-08-01 450 575 Rue St Amable null Quebec Quebec Canada G1R 2006-09-01 451 2512-4th Ave Sw null Calgary Alberta Canada T2P 2G8 2006-12-01 452 55 Lakeshore Blvd East null Toronto Ontario Canada M4B 1V6 2005-09-01 453 6333 Cote Vertu null Montreal Quebec Canada H1Y 2H7 2007-09-01 454 3255 Front Street West null Toronto Ontario Canada M4B 1V6 2007-08-01 455 2550 Signet Drive null Weston Ontario Canada M9V 4W3 2006-08-01 456 6777 Kingsway null Burnaby British Columbia Canada V5H 3Z7 2006-08-01 457 5250-505 Burning St null Vancouver British Columbia Canada V7L 4J4 2006-07-01 458 600 Slater Street null Ottawa Ontario Canada K4B 1S2 2005-07-01 459 25575 The Queensway null Etobicoke Ontario Canada M9W 3P3 2005-08-01 460 2521 McPherson Street null Markham Ontario Canada L3S 3K2 2007-08-01 461 2560 Bay Street null Toronto Ontario Canada M4B 1V7 2006-08-01 462 630 University Avenue null Toronto Ontario Canada M4B 1V7 2005-08-01 463 992 St Clair Ave East null Toronto Ontario Canada M4B 1V7 2005-09-01 464 99, Rue Saint-pierre null Pnot-Rouge Quebec Canada J1E 2T7 2005-07-01 465 25245 Rue Sherbrooke Ouest null Montreal Quebec Canada H1Y 2H5 2007-07-01 466 655-4th Ave S.W. null Calgary Alberta Canada T2P 2G8 2005-08-01 467 25900-700-9th Ave S.W. null Calgary Alberta Canada T2P 2G8 2005-11-01 468 Po Box 83270 null Vancouver British Columbia Canada V7L 4J4 2007-08-01 469 400-25155 West Pender St null Vancouver British Columbia Canada V7L 4J4 2007-09-01 470 2511 Baker Road null Toronto Ontario Canada M4B 1V7 2005-09-01 471 770 Notre Datme Quest Bureau 800 null Montreal Quebec Canada H1Y 2H7 2005-09-01 472 250551 Shellbridge Way null Richmond British Columbia Canada V6B 3P7 2005-09-01 473 595 Burning Street null Vancouver British Columbia Canada V7L 4J4 2005-09-01 474 252345 8810th Avenue null Surrey British Columbia Canada V3T 4W3 2005-09-01 475 2512-410th Avenue S.W. null Calgary Alberta Canada T2P 2G8 2007-09-01 476 258 King Street East null Toronto Ontario Canada M4B 1V7 2005-08-01 477 6th Floor Ferguson Block null Toronto Ontario Canada M4B 1V7 2005-08-01 478 2545 King Street West null Toronto Ontario Canada M4B 1V7 2005-08-01 479 2550 Middlefield Road null Scarborough Ontario Canada M1V 4M2 2005-08-01 480 25 First Canadian Place null Toronto Ontario Canada M4B 1V5 2005-08-01 481 65 Gamelin Street null Hull Quebec Canada 8Y 2006-08-01 482 2500 University Avenue null Toronto Ontario Canada M4B 1V5 2005-07-01 483 955 Green Valley Crescent null Ottawa Ontario Canada K4B 1S1 2007-09-01 484 2510 Crew Court null Montreal Quebec Canada H1Y 2H8 2008-06-01 485 9900-6400 Boul, Taschereau null Brossard Quebec Canada J4Z 1R4 2006-07-01 486 600 Boul. Rene-levesque Ouest null Montreal Quebec Canada H1Y 2H7 2006-07-01 487 9259 - 1110th Avenue Sw null Calgary Alberta Canada T2P 2G8 2006-09-01 488 45259 Canada Way null Burnaby British Columbia Canada V5G 4S4 2007-08-01 489 9960 King Street E. null Toronto Ontario Canada M4B 1V5 2007-08-01 490 9979 Bayview Drive null Barrie Ontario Canada L4N 2006-08-01 491 20225 Lansing Ave 

Processing table: /mnt/silver/SalesLT/Customer
Successfully processed and saved: /mnt/gold/SalesLT/Customer/


customer_id name_style title first_name middle_name last_name suffix company_name sales_person email_address phone password_hash password_salt date 1 false Mr. Orlando N. Gee null A Bike Store adventure-works\pamela0 orlando0@adventure-works.com 245-555-0173 L/Rlwxzp4w7RWmEgXX+/A7cXaePEPcp+KwQhl2fJL7w= 1KjXYs4= 2005-08-01 2 false Mr. Keith null Harris null Progressive Sports adventure-works\david8 keith0@adventure-works.com 170-555-0127 YPdtRdvqeAhj6wyxEsFdshBDNXxkCXn+CRgbvJItknw= fs1ZGhY= 2006-08-01 3 false Ms. Donna F. Carreras null Advanced Bike Components adventure-works\jillian0 donna0@adventure-works.com 279-555-0130 LNoK27abGQo48gGue3EBV/UrlYSToV0/s87dCRV7uJk= YTNH5Rw= 2005-09-01 4 false Ms. Janet M. Gates null Modular Cycle Systems adventure-works\jillian0 janet1@adventure-works.com 710-555-0173 ElzTpSNbUW1Ut+L5cWlfR7MF6nBZia8WpmGaQPjLOJA= nm7D5e4= 2006-07-01 5 false Mr. Lucy null Harrington null Metropolitan Sports Supply adventure-works\shu0 lucy0@adventure-works.com 828-555-0186 KJqV15wsX3PG8TS5GSddp6LFFVdd3CoRftZM/tP0+R4= cNFKU4w= 2006-09-01 6 false Ms. Rosmarie J. Carroll null Aerobic Exercise Company adventure-works\linda3 rosmarie0@adventure-works.com 244-555-0112 OKT0scizCdIzymHHOtyJKQiC/fCILSooSZ8dQ2Y34VM= ihWf50M= 2007-09-01 7 false Mr. Dominic P. Gash null Associated Bikes adventure-works\shu0 dominic0@adventure-works.com 192-555-0173 ZccoP/jZGQm+Xpzc7RKwDhS11YFNybwcPVRYTSNcnSg= sPoUBSQ= 2006-07-01 10 false Ms. Kathleen M. Garza null Rural Cycle Emporium adventure-works\josé1 kathleen0@adventure-works.com 150-555-0127 Qa3aMCxNbVLGrc0b99KsbQqiVgwYDfHcsK9GZSUxcTM= Ls05W3g= 2006-09-01 11 false Ms. Katherine null Harding null Sharp Bikes adventure-works\josé1 katherine0@adventure-works.com 926-555-0159 uRlorVzDGNJIX9I+ehTlRK+liT4UKRgWhApJgUMC2d4= jpHKbqE= 2005-08-01 12 false Mr. Johnny A. Caprio Jr. Bikes and Motorbikes adventure-works\garrett1 johnny0@adventure-works.com 112-555-0191 jtF9jBoFYeJTaET7x+eJDkd7BzMz15Wo9odbGPBaIak= wVLnvHo= 2006-08-01 16 false Mr. Christopher R. Beck Jr. Bulk Discount Store adventure-works\jae0 christopher1@adventure-works.com 1 (11) 500 555-0132 sKt9daCzEEKWAzivEGPOp8tmaM1R3I+aJfcBjzJRFLo= 8KfYx/4= 2006-09-01 18 false Mr. David J. Liu null Catalog Store adventure-works\michael9 david20@adventure-works.com 440-555-0132 61zeTkO+eI5g8GG0swny8Wp/6GzZMFnT71fnW4lTHNY= c7Ttvv0= 2005-08-01 19 false Mr. John A. Beaver null Center Cycle Shop adventure-works\pamela0 john8@adventure-works.com 521-555-0195 DzbqWX7B3EK5Dub92CKHYSUGKGbZCbrcVDpVe/xyBeI= zXNgrJw= 2007-04-01 20 false Ms. Jean P. Handley null Central Discount Store adventure-works\david8 jean1@adventure-works.com 582-555-0113 o1GVo3vExeNzo0/ctdRGf2eDK3uzTlcUbr18tN+Slf8= uMsvfdo= 2005-09-01 21 false null Jinghao null Liu null Chic Department Stores adventure-works\jillian0 jinghao1@adventure-works.com 928-555-0116 IaD5AeqK9mRiIrJi/etZGVO6EiybLf/oksA2CqrpoJ0= p6pOqKc= 2006-09-01 22 false Ms. Linda E. Burnett null Travel Systems adventure-works\jillian0 linda4@adventure-works.com 121-555-0121 23AwhujCoXYSPiN/B+G8Z9rk36xl35EbdLT7akTMTqU= SmyIPjE= 2005-08-01 23 false Mr. Kerim null Hanif null Bike World adventure-works\shu0 kerim0@adventure-works.com 216-555-0122 d0WSjosAd7Y3XOWjNAkoTClCb50vwPuAawOSI1iosgs= 33g5co8= 2006-09-01 24 false Mr. Kevin null Liu null Eastside Department Store adventure-works\linda3 kevin5@adventure-works.com 926-555-0164 ylTpkIOHKLcjihNjS0j/k10eOHOsWQMNhlbuOCp+UTY= TgZnUOg= 2006-09-01 25 false Mr. Donald L. Blanton null Coalition Bike Company adventure-works\shu0 donald0@adventure-works.com 357-555-0161 pKYDelLBOZMO98GBzhMxBSzzE0gUYKx9dXzYTYNuBgw= jKtOaOw= 2006-09-01 28 false Ms. Jackie E. Blackwell null Commuter Bicycle Store adventure-works\josé1 jackie0@adventure-works.com 972-555-0163 wqhgKfOTfef4Zo3cb6FwsFzvG/yCVstuYh3AuwjRszQ= SZ+r60o= 2007-08-01 29 false Mr. Bryan null Hamilton null Cross-Country Riding Supplies adventure-works\josé1 bryan2@adventure-works.com 344-555-0144 ftRWIIT4oF+if+ddn1ROYXHw6PAooKFX

Processing table: /mnt/silver/SalesLT/CustomerAddress
Successfully processed and saved: /mnt/gold/SalesLT/CustomerAddress/


...

product_model_id product_description_id culture date 1 1199 en 2007-06-01 1 1467 ar 2007-06-01 1 1589 fr 2007-06-01 1 1712 th 2007-06-01 1 1838 he 2007-06-01 1 1965 zh-cht 2007-06-01 2 1210 en 2007-06-01 2 1476 ar 2007-06-01 2 1598 fr 2007-06-01 2 1721 th 2007-06-01 2 1847 he 2007-06-01 2 1974 zh-cht 2007-06-01 3 1195 en 2007-06-01 3 1464 ar 2007-06-01 3 1586 fr 2007-06-01 3 1709 th 2007-06-01 3 1835 he 2007-06-01 3 1961 zh-cht 2007-06-01 4 1194 en 2007-06-01 4 1463 ar 2007-06-01 4 1585 fr 2007-06-01 4 1708 th 2007-06-01 4 1834 he 2007-06-01 4 1960 zh-cht 2007-06-01 5 647 en 2007-06-01 5 1400 ar 2007-06-01 5 1605 fr 2007-06-01 5 1639 th 2007-06-01 5 1768 he 2007-06-01 5 1889 zh-cht 2007-06-01 6 1090 en 2007-06-01 6 1451 ar 2007-06-01 6 1573 fr 2007-06-01 6 1696 th 2007-06-01 6 1827 he 2007-06-01 6 1948 zh-cht 2007-06-01 7 1182 en 2007-06-01 7 1453 ar 2007-06-01 7 1575 fr 2007-06-01 7 1698 th 2007-06-01 7 1829 he 2007-06-01 7 1950 zh-cht 2007-06-01 8 637 en 2007-06-01 8 1397 ar 2007-06-01 8 1514 fr 2007-06-01 8 1636 th 2007-06-01 8 1759 he 2007-06-01 8 1886 zh-cht 2007-06-01 9 1020 en 2007-06-01 9 1449 ar 2007-06-01 9 1571 fr 2007-06-01 9 1694 th 2007-06-01 9 1825 he 2007-06-01 9 1946 zh-cht 2007-06-01 10 1146 en 2007-06-01 10 1452 ar 2007-06-01 10 1574 fr 2007-06-01 10 1697 th 2007-06-01 10 1828 he 2007-06-01 10 1949 zh-cht 2007-06-01 11 1211 en 2007-06-01 11 1477 ar 2007-06-01 11 1599 fr 2007-06-01 11 1722 th 2007-06-01 11 1848 he 2007-06-01 11 1975 zh-cht 2007-06-01 12 1192 en 2007-06-01 12 1461 ar 2007-06-01 12 1583 fr 2007-06-01 12 1706 th 2007-06-01 12 1832 he 2007-06-01 12 1958 zh-cht 2007-06-01 13 1200 en 2007-06-01 13 1468 ar 2007-06-01 13 1590 fr 2007-06-01 13 1713 th 2007-06-01 13 1839 he 2007-06-01 13 1966 zh-cht 2007-06-01 14 644 en 2007-06-01 14 1399 ar 2007-06-01 14 1516 fr 2007-06-01 14 1638 th 2007-06-01 14 1761 he 2007-06-01 14 1888 zh-cht 2007-06-01 15 642 en 2007-06-01 15 1398 ar 2007-06-01 15 1515 fr 2007-06-01 15 1637 th 2007-06-01 15 1760 he 2007-06-01 15 1887 zh-cht 2007-06-01 16 1062 en 2007-06-01 16 1450 ar 2007-06-01 16 1572 fr 2007-06-01 16 1695 th 2007-06-01 16 1826 he 2007-06-01 16 1947 zh-cht 2007-06-01 17 661 en 2007-06-01 17 1401 ar 2007-06-01 17 1517 fr 2007-06-01 17 1640 th 2007-06-01 17 1770 he 2007-06-01 17 1890 zh-cht 2007-06-01 18 1189 en 2007-06-01 18 1459 ar 2007-06-01 18 1581 fr 2007-06-01 18 1704 th 2007-06-01 18 1830 he 2007-06-01 18 1956 zh-cht 2007-06-01 19 168 en 2007-06-01 19 1367 ar 2007-06-01 19 1491 fr 2007-06-01 19 1613 th 2007-06-01 19 1735 he 2007-06-01 19 1862 zh-cht 2007-06-01 20 128 en 2007-06-01 20 1366 ar 2007-06-01 20 1490 fr 2007-06-01 20 1612 th 2007-06-01 20 1734 he 2007-06-01 20 1861 zh-cht 2007-06-01 21 88 en 2007-06-01 21 1365 ar 2007-06-01 21 1489 fr 2007-06-01 21 1611 th 2007-06-01 21 1733 he 2007-06-01 21 1860 zh-cht 2007-06-01 22 64 en 2007-06-01 22 1364 ar 2007-06-01 22 1488 fr 2007-06-01 22 1610 th 2007-06-01 22 1732 he 2007-06-01 22 1859 zh-cht 2007-06-01 23 8 en 2007-06-01 23 1363 ar 2007-06-01 23 1487 fr 2007-06-01 23 1609 th 2007-06-01 23 1731 he 2007-06-01 23 1858 zh-cht 2007-06-01 24 1208 en 2007-06-01 24 1474 ar 2007-06-01 24 1596 fr 2007-06-01 24 1719 th 2007-06-01 24 1845 he 2007-06-01 24 1972 zh-cht 2007-06-01 25 457 en 2007-06-01 25 1377 ar 2007-06-01 25 1501 fr 2007-06-01 25 1623 th 2007-06-01 25 1745 he 2007-06-01 25 1872 zh-cht 2007-06-01 26 409 en 2007-06-01 26 1376 ar 2007-06-01 26 1500 fr 2007-06-01 26 1622 th 2007-06-01 26 1744 he 2007-06-01 26 1871 zh-cht 2007-06-01 27 376 en 2007-06-01 27 1375 ar 2007-06-01 27 1499 fr 2007-06-01 27 1621 th 2007-06-01 27 1743 he 2007-06-01 27 1870 zh-cht 2007-06-01 28 337 en 2007-06-01 28 1373 ar 2007-06-01 28 1497 fr 2007-06-01 28 1619 th 2007-06-01 28 1741 he 2007-06-01 28 1868 zh-cht 2007-06-01 29 320 en 2007-06-01 29 1371 ar 2007-06-01 29 1495 fr 2007-06-01 29 1617 th 2007-06-01 29 1739 he 2007-06-01 29 1866 zh-cht 2007-06-01 30 249 en 2007-06-01 30 1370 ar 2007-06-01 30 1494 fr 2007-06-01 30 1616 th 

Processing table: /mnt/silver/SalesLT/SalesOrderDetail
Successfully processed and saved: /mnt/gold/SalesLT/SalesOrderDetail/


sales_order_id sales_order_detail_id order_qty product_id unit_price unit_price_discount line_total date 71774 110562 1 836 356.8980 0.0000 356.898000 2008-06-01 71774 110563 1 822 356.8980 0.0000 356.898000 2008-06-01 71776 110567 1 907 63.9000 0.0000 63.900000 2008-06-01 71780 110616 4 905 218.4540 0.0000 873.816000 2008-06-01 71780 110617 2 983 461.6940 0.0000 923.388000 2008-06-01 71780 110618 6 988 112.9980 0.4000 406.792800 2008-06-01 71780 110619 2 748 818.7000 0.0000 1637.400000 2008-06-01 71780 110620 1 990 323.9940 0.0000 323.994000 2008-06-01 71780 110621 1 926 149.8740 0.0000 149.874000 2008-06-01 71780 110622 1 743 809.7600 0.0000 809.760000 2008-06-01 71780 110623 4 782 1376.9940 0.0000 5507.976000 2008-06-01 71780 110624 2 918 158.4300 0.0000 316.860000 2008-06-01 71780 110625 4 780 1391.9940 0.0000 5567.976000 2008-06-01 71780 110626 1 937 48.5940 0.0000 48.594000 2008-06-01 71780 110627 6 867 41.9940 0.0000 251.964000 2008-06-01 71780 110628 1 985 112.9980 0.4000 67.798800 2008-06-01 71780 110629 2 989 323.9940 0.0000 647.988000 2008-06-01 71780 110630 3 991 323.9940 0.0000 971.982000 2008-06-01 71780 110631 1 992 323.9940 0.0000 323.994000 2008-06-01 71780 110632 2 993 323.9940 0.0000 647.988000 2008-06-01 71780 110633 2 984 112.9980 0.4000 135.597600 2008-06-01 71780 110634 3 986 112.9980 0.4000 203.396400 2008-06-01 71780 110635 3 987 112.9980 0.4000 203.396400 2008-06-01 71780 110636 2 981 461.6940 0.0000 923.388000 2008-06-01 71780 110637 3 982 461.6940 0.0000 1385.082000 2008-06-01 71780 110638 5 783 1376.9940 0.0000 6884.970000 2008-06-01 71780 110639 3 809 37.1520 0.0000 111.456000 2008-06-01 71780 110640 1 810 72.1620 0.0000 72.162000 2008-06-01 71780 110641 2 935 24.2940 0.0000 48.588000 2008-06-01 71780 110642 1 925 149.8740 0.0000 149.874000 2008-06-01 71780 110643 7 869 41.9940 0.0000 293.958000 2008-06-01 71780 110644 1 880 32.9940 0.0000 32.994000 2008-06-01 71782 110667 3 714 29.9940 0.0000 89.982000 2008-06-01 71782 110668 3 956 1430.4420 0.0000 4291.326000 2008-06-01 71782 110669 1 954 1430.4420 0.0000 1430.442000 2008-06-01 71782 110670 10 712 5.3940 0.0000 53.940000 2008-06-01 71782 110671 10 877 4.7700 0.0000 47.700000 2008-06-01 71782 110672 4 996 72.8940 0.0000 291.576000 2008-06-01 71782 110673 6 864 38.1000 0.0000 228.600000 2008-06-01 71782 110674 3 876 72.0000 0.0000 216.000000 2008-06-01 71782 110675 4 959 445.4100 0.0000 1781.640000 2008-06-01 71782 110676 4 971 728.9100 0.0000 2915.640000 2008-06-01 71782 110677 6 870 2.9940 0.0000 17.964000 2008-06-01 71782 110678 1 978 445.4100 0.0000 445.410000 2008-06-01 71782 110679 1 955 1430.4420 0.0000 1430.442000 2008-06-01 71782 110680 1 948 63.9000 0.0000 63.900000 2008-06-01 71782 110681 1 994 32.3940 0.0000 32.394000 2008-06-01 71782 110682 1 970 728.9100 0.0000 728.910000 2008-06-01 71782 110683 2 966 1430.4420 0.0000 2860.884000 2008-06-01 71782 110684 4 884 32.3940 0.0000 129.576000 2008-06-01 71782 110685 2 885 602.3460 0.0000 1204.692000 2008-06-01 71782 110686 8 715 29.9940 0.0000 239.952000 2008-06-01 71782 110687 3 894 72.8760 0.0000 218.628000 2008-06-01 71782 110688 1 957 1430.4420 0.0000 1430.442000 2008-06-01 71782 110689 2 967 1430.4420 0.0000 2860.884000 2008-06-01 71782 110690 7 708 20.9940 0.0000 146.958000 2008-06-01 71782 110691 3 961 445.4100 0.0000 1336.230000 2008-06-01 71782 110692 2 979 445.4100 0.0000 890.820000 2008-06-01 71782 110693 2 958 445.4100 0.0000 890.820000 2008-06-01 71782 110694 1 963 445.4100 0.0000 445.410000 2008-06-01 71782 110695 1 953 728.9100 0.0000 728.910000 2008-06-01 71782 110696 2 968 1430.4420 0.0000 2860.884000 2008-06-01 71782 110697 2 951 242.9940 0.0000 485.988000 2008-06-01 71782 110698 2 945 54.8940 0.0000 109.788000 2008-06-01 71782 110699 1 916 31.5840 0.0000 31.584000 2008-06-01 71782 110700 1 886 200.0520 0.0000 200.052000 2008-06-01 71782 110701 1 892 602.3460 0.0000 602.346000 2008-06-01 71782 110702 1 893 602.3460 0.0000 602.346000 2008-06-01 71782 110703 6 881 32.3940 0.0

Processing table: /mnt/silver/SalesLT/SalesOrderHeader
Successfully processed and saved: /mnt/gold/SalesLT/SalesOrderHeader/


sales_order_id revision_number order_date due_date ship_date status online_order_flag sales_order_number purchase_order_number account_number customer_id ship_to_address_id bill_to_address_id ship_method credit_card_approval_code sub_total tax_amt freight total_due comment date 71774 2 2008-06-01T00:00:00Z 2008-06-13T00:00:00Z 2008-06-08T00:00:00Z 5 false SO71774 PO348186287 10-4020-000609 29847 1092 1092 CARGO TRANSPORT 5 null 880.3484 70.4279 22.0087 972.7850 null 2008-06-08 71776 2 2008-06-01T00:00:00Z 2008-06-13T00:00:00Z 2008-06-08T00:00:00Z 5 false SO71776 PO19952192051 10-4020-000106 30072 640 640 CARGO TRANSPORT 5 null 78.8100 6.3048 1.9703 87.0851 null 2008-06-08 71780 2 2008-06-01T00:00:00Z 2008-06-13T00:00:00Z 2008-06-08T00:00:00Z 5 false SO71780 PO19604173239 10-4020-000340 30113 653 653 CARGO TRANSPORT 5 null 38418.6895 3073.4952 960.4672 42452.6519 null 2008-06-08 71782 2 2008-06-01T00:00:00Z 2008-06-13T00:00:00Z 2008-06-08T00:00:00Z 5 false SO71782 PO19372114749 10-4020-000582 29485 1086 1086 CARGO TRANSPORT 5 null 39785.3304 3182.8264 994.6333 43962.7901 null 2008-06-08 71783 2 2008-06-01T00:00:00Z 2008-06-13T00:00:00Z 2008-06-08T00:00:00Z 5 false SO71783 PO19343113609 10-4020-000024 29957 992 992 CARGO TRANSPORT 5 null 83858.4261 6708.6741 2096.4607 92663.5609 null 2008-06-08 71784 2 2008-06-01T00:00:00Z 2008-06-13T00:00:00Z 2008-06-08T00:00:00Z 5 false SO71784 PO19285135919 10-4020-000448 29736 659 659 CARGO TRANSPORT 5 null 108561.8317 8684.9465 2714.0458 119960.8240 null 2008-06-08 71796 2 2008-06-01T00:00:00Z 2008-06-13T00:00:00Z 2008-06-08T00:00:00Z 5 false SO71796 PO17052159664 10-4020-000420 29660 1058 1058 CARGO TRANSPORT 5 null 57634.6342 4610.7707 1440.8659 63686.2708 null 2008-06-08 71797 2 2008-06-01T00:00:00Z 2008-06-13T00:00:00Z 2008-06-08T00:00:00Z 5 false SO71797 PO16501134889 10-4020-000142 29796 642 642 CARGO TRANSPORT 5 null 78029.6898 6242.3752 1950.7422 86222.8072 null 2008-06-08 71815 2 2008-06-01T00:00:00Z 2008-06-13T00:00:00Z 2008-06-08T00:00:00Z 5 false SO71815 PO13021155785 10-4020-000276 30089 1034 1034 CARGO TRANSPORT 5 null 1141.5782 91.3263 28.5395 1261.4440 null 2008-06-08 71816 2 2008-06-01T00:00:00Z 2008-06-13T00:00:00Z 2008-06-08T00:00:00Z 5 false SO71816 PO12992180445 10-4020-000295 30027 1038 1038 CARGO TRANSPORT 5 null 3398.1659 271.8533 84.9541 3754.9733 null 2008-06-08 71831 2 2008-06-01T00:00:00Z 2008-06-13T00:00:00Z 2008-06-08T00:00:00Z 5 false SO71831 PO10295111084 10-4020-000322 30019 652 652 CARGO TRANSPORT 5 null 2016.3408 161.3073 50.4085 2228.0566 null 2008-06-08 71832 2 2008-06-01T00:00:00Z 2008-06-13T00:00:00Z 2008-06-08T00:00:00Z 5 false SO71832 PO10353140756 10-4020-000088 29922 639 639 CARGO TRANSPORT 5 null 35775.2113 2862.0169 894.3803 39531.6085 null 2008-06-08 71845 2 2008-06-01T00:00:00Z 2008-06-13T00:00:00Z 2008-06-08T00:00:00Z 5 false SO71845 PO2697119362 10-4020-000187 29938 1020 1020 CARGO TRANSPORT 5 null 41622.0511 3329.7641 1040.5513 45992.3665 null 2008-06-08 71846 2 2008-06-01T00:00:00Z 2008-06-13T00:00:00Z 2008-06-08T00:00:00Z 5 false SO71846 PO2378131604 10-4020-000635 30102 669 669 CARGO TRANSPORT 5 null 2453.7645 196.3012 61.3441 2711.4098 null 2008-06-08 71856 2 2008-06-01T00:00:00Z 2008-06-13T00:00:00Z 2008-06-08T00:00:00Z 5 false SO71856 PO16530177647 10-4020-000601 30033 1090 1090 CARGO TRANSPORT 5 null 602.1946 48.1756 15.0549 665.4251 null 2008-06-08 71858 2 2008-06-01T00:00:00Z 2008-06-13T00:00:00Z 2008-06-08T00:00:00Z 5 false SO71858 PO16153112278 10-4020-000186 29653 1019 1019 CARGO TRANSPORT 5 null 13823.7083 1105.8967 345.5927 15275.1977 null 2008-06-08 71863 2 2008-06-01T00:00:00Z 2008-06-13T00:00:00Z 2008-06-08T00:00:00Z 5 false SO71863 PO16124166561 10-4020-000649 29975 1098 1098 CARGO TRANSPORT 5 null 3324.2759 265.9421 83.1069 3673.3249 null 2008-06-08 71867 2 2008-06-01T00:00:00Z 2008-06-13T00:00:00Z 2008-06-08T00:00:00Z 5 false SO71867 PO13050111529 10-4020-000160 29644 643 643 CARGO TRANSPORT 5 null 1059.3100 84.7448 26.48